# RSA Digital Signatures

RSA can also be used for digital signatures. Rather than encrypting a message to hide its content, digital signatures allow Bob to prove that a message really came from him. Alice verifies the signature with Bob’s public key, ensuring the message’s authenticity and integrity, while Eve cannot forge the signature without Bob’s private key.

## Communication Flow

```
Bob                                Alice                      Eve
---                                -----                      ---
[Message m]
   |                             
Hash and sign with
private key (d)
   |                             
Signature (sig)                 |
   |----> Send (m, sig) ------>  |  Receives (m, sig)
   |                             |  Sees both m and sig
   |                             |  Verify signature with Bob's
   |                             | public key (N, e)
   |                             |

```
Eve can not forge the signature.
### Key Generation
- **Public Key:** (N, e)
- **Private Key:** (N, d)

### Signing (Performed by Bob)
1. Compute the hash of the message: `h = H(m)`.
2. Generate the signature: `s = h^d mod N`.

### Verification (Performed by Alice)
1. Compute the verification value: `v = s^e mod N`.
2. Check if `v == h`. If they match, the signature is valid.

## Parameters

- **N:** Modulus (public) — the product of two large primes.
- **e:** Public exponent (public) — commonly set to `65537`.
- **d:** Private exponent (secret).
- **h:** Hash of the message.
- **s:** Signature value.
- **m:** Message.

## Security Notes

- **Hashing:** Always hash the message before signing (e.g., SHA-256).
- **Padding:** Use secure padding for RSA signatures (RSA-PSS is recommended).
- **Never Sign Raw Messages:** Always sign the hash of the message.
- **Key Size:** Use RSA keys of at least 2048 bits for adequate security.

## Where It’s Used

- **TLS Certificates:** Websites use RSA signatures to prove their identity.
- **Software Signing:** Applications and code packages are signed to ensure authenticity.
- **Email Signing:** PGP and S/MIME employ RSA signatures to secure email communications.

In [1]:
import hashlib

# --- Key generation (toy RSA, same as before) ---
p = 61
q = 53
N = p * q
phi = (p-1)*(q-1)
e = 17

def inv_mod(a, m):
    def egcd(a, b):
        if b == 0:
            return (1, 0, a)
        x1, y1, g = egcd(b, a % b)
        return (y1, x1 - (a // b) * y1, g)
    x, y, g = egcd(a, m)
    if g != 1:
        raise Exception("No inverse")
    return x % m

d = inv_mod(e, phi)

print("Public key (N, e):", (N, e))
print("Private key (N, d):", (N, d))

# --- Signing (Bob) ---
message = b"Hello Alice, this is Bob."
# Hash the message (toy: use SHA-256 and convert to int)
h = int.from_bytes(hashlib.sha256(message).digest(), "big") % N
signature = pow(h, d, N)

print("Message:", message)
print("Signature:", signature)

# --- Verifying (Alice) ---
h_check = pow(signature, e, N)  # recover hash from signature
h_expected = int.from_bytes(hashlib.sha256(message).digest(), "big") % N

if h_check == h_expected:
    print("Signature valid, message is from Bob.")
else:
    print("Signature invalid, message may be tampered.")


Public key (N, e): (3233, 17)
Private key (N, d): (3233, 2753)
Message: b'Hello Alice, this is Bob.'
Signature: 1206
Signature valid, message is from Bob.
